In [42]:
import extract_and_transform as et
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
import os
import joblib

In [43]:
X_train,X_test,y_train,y_test=et.training_data()

In [44]:
X_train

,cement,blast_furnace_slage,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age
995,158.6,148.9,116.0,175.1,15.0,953.3,719.7,28
507,424.0,22.0,132.0,178.0,8.5,822.0,750.0,28
334,275.1,0.0,121.4,159.5,9.9,1053.6,777.5,3
848,252.0,97.0,76.0,194.0,8.0,835.0,821.0,28
294,168.9,42.2,124.3,158.3,10.8,1080.8,796.2,3
...,...,...,...,...,...,...,...,...
87,286.3,200.9,0.0,144.7,11.2,1004.6,803.7,3
330,246.8,0.0,125.1,143.3,12.0,1086.8,800.9,14
466,190.3,0.0,125.2,166.6,9.9,1079.0,798.9,100
121,475.0,118.8,0.0,181.1,8.9,852.1,781.5,28


In [31]:
scaler=StandardScaler()
scaler.fit(X_train)
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [32]:
X_train_scaled

array([[-1.16087734,  0.85738747,  0.9824489 , ..., -0.25746155,
        -0.64752011, -0.27567315],
       [ 1.30862304, -0.60249189,  1.23259821, ..., -1.92694961,
        -0.2731482 , -0.27567315],
       [-0.0768653 , -0.85558366,  1.06687429, ...,  1.01785948,
         0.06662828, -0.68931339],
       ...,
       [-0.86591441, -0.85558366,  1.12628475, ...,  1.34082214,
         0.33103616,  0.91561074],
       [ 1.78316909,  0.51111191, -0.83113361, ..., -1.54422615,
         0.11605031, -0.27567315],
       [ 0.28509237, -0.85558366,  0.9355459 , ..., -0.6172979 ,
         0.13458358, -0.27567315]])

In [45]:
feature_names=["cement","blast_furnace_slage","fly_ash","water","superplasticizer","coarse_aggregate","fine_aggregate","age"]

In [46]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

In [47]:
max_depth = [45]#[int(x) for x in np.linspace(1,201,10)]
max_depth.append(None)
param_grid={'bootstrap':[False],
            'max_depth':max_depth,
            'max_features':['auto','log2'],
            'n_estimators': [156]#[int(x) for x in np.linspace(1, 201, 10)] }

In [48]:
rfr=RandomForestRegressor(random_state=1)

In [49]:
g_search=GridSearchCV(estimator=rfr,param_grid=param_grid,cv=5,n_jobs=1,verbose=0,return_train_score=True)

In [50]:
g_search.fit(X_train,y_train)
g_search.best_params_

{'bootstrap': False,
 'max_depth': 45,
 'max_features': 'log2',
 'n_estimators': 156}

In [51]:
print(g_search.score(X_test,y_test))

0.8917394743932537


In [52]:
predictions=g_search.predict(X_test)
mse_rf=mean_squared_error(y_test,predictions)
mae_rf=mean_absolute_error(y_test,predictions)
print('Mean squared error from random forest regression: ', mse_rf)
print('Mean absolute error from random forest regression: ', mae_rf)
import pandas as pd
feature_imp=pd.Series(g_search.best_estimator_.feature_importances_,index=feature_names).sort_values()

Mean squared error from random forest regression:  27.896236020482345
Mean absolute error from random forest regression:  3.442603881420634


In [53]:
feature_imp

fine_aggregate         0.050848
coarse_aggregate       0.051582
fly_ash                0.054364
blast_furnace_slage    0.064093
superplasticizer       0.071039
water                  0.122931
cement                 0.238894
age                    0.346249
dtype: float64

In [54]:
def print_predict(y_test,predictions):
    for i in range(len(predictions)):
        act=y_test[i]
        pred=round(predictions[i],2)
        delta=round(act-pred,2)
        print(f"Actual: {act}, Prediction: {pred}, Delta: {delta}")

In [55]:

g_search.score(X_test,y_test)

0.8917394743932537

In [56]:
print_predict(y_test,predictions)

Actual: 52.91, Prediction: 51.98, Delta: 0.93
Actual: 55.9, Prediction: 39.4, Delta: 16.5
Actual: 74.5, Prediction: 72.21, Delta: 2.29
Actual: 35.3, Prediction: 35.3, Delta: 0.0
Actual: 10.54, Prediction: 10.99, Delta: -0.45
Actual: 44.28, Prediction: 44.87, Delta: -0.59
Actual: 23.69, Prediction: 23.73, Delta: -0.04
Actual: 45.37, Prediction: 47.76, Delta: -2.39
Actual: 37.4, Prediction: 35.09, Delta: 2.31
Actual: 48.85, Prediction: 43.56, Delta: 5.29
Actual: 25.1, Prediction: 39.79, Delta: -14.69
Actual: 10.73, Prediction: 14.92, Delta: -4.19
Actual: 40.2, Prediction: 43.48, Delta: -3.28
Actual: 49.8, Prediction: 34.72, Delta: 15.08
Actual: 33.01, Prediction: 25.13, Delta: 7.88
Actual: 21.92, Prediction: 24.61, Delta: -2.69
Actual: 40.06, Prediction: 38.73, Delta: 1.33
Actual: 19.35, Prediction: 17.3, Delta: 2.05
Actual: 37.17, Prediction: 36.91, Delta: 0.26
Actual: 33.09, Prediction: 33.18, Delta: -0.09
Actual: 31.38, Prediction: 33.87, Delta: -2.49
Actual: 37.96, Prediction: 38.07,

In [57]:
# save
joblib.dump(g_search, "random_forest_regression.joblib")

['random_forest_regression.joblib']

In [25]:
# load, no need to initialize the loaded_rf
rfr_model = joblib.load("random_forest_regression.joblib")

In [58]:
data=np.array([10,10,10,10,10,10,10,10]).reshape(1,-1)
rfr_model.predict(data)

array([28.42666667])

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

In [ ]:
n_estimators = [int(x) for x in np.linspace(5 , 150, 145)] 

In [ ]:
max_depth = [int(x) for x in np.linspace(1,200,200)]
max_depth.append(None)

In [ ]:
max_depth

In [ ]:
bootstrap=[True,False]

In [ ]:
random_grid={'n_estimators':n_estimators,'max_features':['auto','log2'],'max_depth':max_depth,'bootstrap':bootstrap}

In [ ]:
print(random_grid)

In [ ]:
random_search=RandomizedSearchCV(estimator=rfr,
                                 param_distributions=random_grid,
                                 n_iter=50,scoring='neg_mean_absolute_error',
                                cv=3,verbose=0,random_state=42,
                                 n_jobs=1,return_train_score=True)

In [ ]:
random_search.fit(X_train,y_train)

In [ ]:
random_search.best_params_

In [ ]:
random_search.score(X_test,y_test)

In [ ]:
print_predict(y_test,random_search.predict(X_test))